# IBM Applied Data Science Capstone Course by Coursera¶


## Week 5 Final Report


### Opening a New Vegan/Vegetarian Venue in Austin, Texas



In [3]:
!pip install geocoder
!pip install folium==0.5.0
!pip install beautifulsoup4


     |████████████████████████████████| 102kB 8.1MB/s ta 0:00:011
     |████████████████████████████████| 81kB 8.0MB/s eta 0:00:011
  Stored in directory: /home/dsxuser/.cache/pip/wheels/f8/98/ff/954791afc47740d554f0d9e5885fa09dd60c2265d42578e665
Successfully built folium


In [11]:
from bs4 import BeautifulSoup
 
import geocoder

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import sqlite3 

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# Scrape of Wiki Page

In [12]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighborhoods_in_Austin,_Texas").text

In [13]:
soup = BeautifulSoup(data, 'html.parser')

In [14]:
neighborhoodList = []

In [15]:
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [51]:
aus_df = pd.DataFrame({"Neighborhood": neighborhoodList})

aus_df.head()

,Neighborhood
0,"Allandale, Austin, Texas"
1,"Anderson Mill, Austin, Texas"
2,List of Austin neighborhoods
3,"Barton Hills, Austin, Texas"
4,"Belterra, Austin, Texas"


In [52]:
aus_df

,Neighborhood
0,"Allandale, Austin, Texas"
1,"Anderson Mill, Austin, Texas"
2,List of Austin neighborhoods
3,"Barton Hills, Austin, Texas"
4,"Belterra, Austin, Texas"
5,"Blackland, Austin, Texas"
6,"Bouldin Creek, Austin, Texas"
7,"Bremond Block Historic District (Austin, Texas)"
8,"Brentwood, Austin, Texas"
9,"Brown Herman, Austin"


In [53]:
aus_df.shape

(61, 1)

# Get Coordinates

In [54]:
def get_latlng(neighborhood):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Austin, Texas'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [55]:
coords = [ get_latlng(neighborhood) for neighborhood in aus_df["Neighborhood"].tolist() ]
coords

[[30.335574180726088, -97.74459899532893],
 [30.449935223935313, -97.81710542954886],
 [32.784380000000056, -96.65740999999997],
 [30.255510000000072, -97.78348999999997],
 [30.194067986694172, -97.98071077543959],
 [30.268790000000024, -97.73991999999998],
 [30.252417299995507, -97.75775257523338],
 [30.26759000000004, -97.74298999999996],
 [30.337585549871914, -97.7288880041651],
 [30.354300036725732, -97.66736990197904],
 [30.305032357606656, -97.7541918938671],
 [30.310074391450033, -97.76038284140986],
 [30.30648989683839, -97.79960763785327],
 [30.27147000000008, -97.72550999999999],
 [30.22575373418368, -98.03046438664214],
 [30.281330000000025, -97.76242999999994],
 [30.267634924862623, -97.74297819864873],
 [30.395552691781557, -97.6591481628773],
 [30.224813437872953, -97.91133721066102],
 [30.260163451268077, -97.7557860902914],
 [30.188435400546854, -97.75457249539305],
 [30.268790000000024, -97.73991999999998],
 [30.254640337612553, -97.7175653437737],
 [30.21666866488036,

In [56]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
aus_df['Latitude'] = df_coords['Latitude']
aus_df['Longitude'] = df_coords['Longitude']
print(aus_df.shape)
aus_df

(61, 3)


,Neighborhood,Latitude,Longitude
0,"Allandale, Austin, Texas",30.335574,-97.744599
1,"Anderson Mill, Austin, Texas",30.449935,-97.817105
2,List of Austin neighborhoods,32.784380,-96.657410
3,"Barton Hills, Austin, Texas",30.255510,-97.783490
4,"Belterra, Austin, Texas",30.194068,-97.980711
5,"Blackland, Austin, Texas",30.268790,-97.739920
6,"Bouldin Creek, Austin, Texas",30.252417,-97.757753
7,"Bremond Block Historic District (Austin, Texas)",30.267590,-97.742990
8,"Brentwood, Austin, Texas",30.337586,-97.728888
9,"Brown Herman, Austin",30.354300,-97.667370


In [57]:
aus_df.to_csv("aus_df.csv", index=False)


In [58]:
address = 'Austin, Texas'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of  Austin, Texas {}, {}.'.format(latitude, longitude))

The geograpical coordinate of  Austin, Texas 30.2711286, -97.7436995.


# Create Map of Austin

In [59]:
map_aus = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(aus_df['Latitude'], aus_df['Longitude'], aus_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_aus)  
    
map_aus

In [61]:
aus_df.drop([2], inplace = True) 

map_aus.save('map_kl.html')


# Explore Neighborhoods in Austin

In [62]:
CLIENT_ID = '3B4ENI5STNPLFHIU2MHHQEHOLUPW3D11U5KQVW0FHRDJRPFG' # your Foursquare ID
CLIENT_SECRET = '2CD4155PTAS13C1ESJ3HOIJJLY5LMXZIO4ZA5OOC3MJJ2SWS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3B4ENI5STNPLFHIU2MHHQEHOLUPW3D11U5KQVW0FHRDJRPFG
CLIENT_SECRET:2CD4155PTAS13C1ESJ3HOIJJLY5LMXZIO4ZA5OOC3MJJ2SWS


## let's get the top 100 venues that are within a radius of 2000 meters.


In [63]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(aus_df['Latitude'], aus_df['Longitude'], aus_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [64]:
venues_df = pd.DataFrame(venues)

venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(4654, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"Allandale, Austin, Texas",30.335574,-97.744599,Amy's Ice Creams,30.332320,-97.740266,Ice Cream Shop
1,"Allandale, Austin, Texas",30.335574,-97.744599,Asahi Imports,30.336347,-97.739225,Supermarket
2,"Allandale, Austin, Texas",30.335574,-97.744599,Sap's Fine Thai Cuisine,30.333477,-97.740875,Thai Restaurant
3,"Allandale, Austin, Texas",30.335574,-97.744599,The Peached Tortilla,30.330815,-97.740227,Asian Restaurant
4,"Allandale, Austin, Texas",30.335574,-97.744599,Barley Swine,30.341256,-97.738458,New American Restaurant


## Let's check how many venues were returned for each neighorhood



In [65]:
venues_df.groupby(["Neighborhood"]).count()


,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"Allandale, Austin, Texas",96,96,96,96,96,96
"Anderson Mill, Austin, Texas",77,77,77,77,77,77
"Barton Hills, Austin, Texas",100,100,100,100,100,100
"Belterra, Austin, Texas",25,25,25,25,25,25
"Blackland, Austin, Texas",100,100,100,100,100,100
"Bouldin Creek, Austin, Texas",100,100,100,100,100,100
"Bremond Block Historic District (Austin, Texas)",100,100,100,100,100,100
"Brentwood, Austin, Texas",100,100,100,100,100,100
"Brown Herman, Austin",25,25,25,25,25,25


## Let's find out how many unique categories there are


In [66]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))


There are 298 uniques categories.


In [67]:
venues_df['VenueCategory'].unique()


array(['Ice Cream Shop', 'Supermarket', 'Thai Restaurant',
       'Asian Restaurant', 'New American Restaurant', 'Pizza Place',
       'Gift Shop', 'Food Truck', 'Bar', 'Burger Joint', 'Flower Shop',
       'Food & Drink Shop', 'Pet Store', 'Sandwich Place',
       'Mexican Restaurant', 'Coffee Shop', 'Park', 'Lounge',
       'Falafel Restaurant', 'Pharmacy', 'Dive Bar',
       'Health & Beauty Service', 'Fried Chicken Joint', 'Taco Place',
       'Gym', 'Korean Restaurant', 'French Restaurant', 'Bakery',
       'Japanese Restaurant', 'American Restaurant', 'Hobby Shop',
       'Juice Bar', 'Cosmetics Shop', 'Donut Shop',
       'Thrift / Vintage Store', 'Cemetery', 'Recreation Center',
       'Bowling Alley', "Doctor's Office", 'Restaurant',
       'Furniture / Home Store', 'Italian Restaurant', 'Yoga Studio',
       'Sushi Restaurant', 'Spa', 'Playground',
       'Vegetarian / Vegan Restaurant', 'Baseball Field',
       'Shipping Store', 'BBQ Joint', 'Gas Station', 'Bank',
       'Co

# Analzye Neighborhoods in Austin

In [68]:
aus_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

aus_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

fixed_columns = [aus_onehot.columns[-1]] + list(aus_onehot.columns[:-1])
aus_onehot = aus_onehot[fixed_columns]

print(aus_onehot.shape)
aus_onehot.head()

(4654, 299)


,Neighborhoods,Accessories Store,Adult Boutique,Advertising Agency,African Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Capitol Building,Caribbean Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Basketball Court,College Bookstore,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Escape Room,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fireworks Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indoor Play Area,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lighthouse,Lighting Store,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Supply Store,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pawn Shop,Pedestrian Plaza,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Public Art,RV Park,Rafting,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Student Center,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Vacation Rental,Vegetarian / Vegan Restaurant,Vehicle Inspection S

## Group each neighborhood and take the frequency of occurrence of each category

In [69]:
aus_grouped = aus_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(aus_grouped.shape)
aus_grouped

(60, 299)


,Neighborhoods,Accessories Store,Adult Boutique,Advertising Agency,African Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Capitol Building,Caribbean Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Basketball Court,College Bookstore,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Escape Room,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fireworks Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indoor Play Area,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lighthouse,Lighting Store,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Supply Store,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pawn Shop,Pedestrian Plaza,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Public Art,RV Park,Rafting,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Student Center,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Vacation Rental,Vegetarian / Vegan Restaurant,Vehicle Inspection S

In [70]:
len(aus_grouped[aus_grouped["Vegetarian / Vegan Restaurant"] > 0])


21

## Create a new DataFrame for Vegetarian / Vegan Restaurant data only



In [71]:
aus_vegan = aus_grouped[["Neighborhoods","Vegetarian / Vegan Restaurant"]]
aus_vegan.head()


,Neighborhoods,Vegetarian / Vegan Restaurant
0,"Allandale, Austin, Texas",0.010417
1,"Anderson Mill, Austin, Texas",0.000000
2,"Barton Hills, Austin, Texas",0.010000
3,"Belterra, Austin, Texas",0.000000
4,"Blackland, Austin, Texas",0.000000


# Cluster Neigborhoods

In [73]:
kclusters = 3

aus_clustering = aus_vegan.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(aus_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 2, 0, 0, 1, 0, 0, 0, 2], dtype=int32)

In [74]:
aus_merged = aus_vegan.copy()

aus_merged["Cluster Labels"] = kmeans.labels_

In [75]:
aus_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
aus_merged.head()

,Neighborhood,Vegetarian / Vegan Restaurant,Cluster Labels
0,"Allandale, Austin, Texas",0.010417,2
1,"Anderson Mill, Austin, Texas",0.000000,0
2,"Barton Hills, Austin, Texas",0.010000,2
3,"Belterra, Austin, Texas",0.000000,0
4,"Blackland, Austin, Texas",0.000000,0


In [76]:
aus_merged = aus_merged.join(aus_df.set_index("Neighborhood"), on="Neighborhood")

print(aus_merged.shape)
aus_merged.head() 

(60, 5)


,Neighborhood,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
0,"Allandale, Austin, Texas",0.010417,2,30.335574,-97.744599
1,"Anderson Mill, Austin, Texas",0.000000,0,30.449935,-97.817105
2,"Barton Hills, Austin, Texas",0.010000,2,30.255510,-97.783490
3,"Belterra, Austin, Texas",0.000000,0,30.194068,-97.980711
4,"Blackland, Austin, Texas",0.000000,0,30.268790,-97.739920


In [77]:
print(aus_merged.shape)
aus_merged.sort_values(["Cluster Labels"], inplace=True)
aus_merged

(60, 5)


,Neighborhood,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
29,"Montopolis, Austin, Texas",0.000000,0,30.205513,-97.716575
22,"East Riverside-Oltorf, Austin, Texas",0.000000,0,30.216669,-97.685525
23,Easton Park,0.000000,0,30.180235,-97.620692
24,"French Place, Austin Texas",0.000000,0,30.286838,-97.719248
25,"Galindo, Austin, Texas",0.000000,0,30.228714,-97.695736
26,"Granada Hills, Austin, Texas",0.000000,0,30.220538,-97.736842
45,"South River City, Austin, Texas",0.000000,0,30.254720,-97.744580
28,"Hyde Park (Austin, Texas)",0.000000,0,30.179040,-97.799700
47,Swedish Hill Historic District,0.000000,0,32.778280,-96.794850
44,"South Lamar, Austin, Texas",0.000000,0,30.232261,-97.804078


In [78]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(aus_merged['Latitude'], aus_merged['Longitude'], aus_merged['Neighborhood'], aus_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [79]:
map_clusters.save('map_clusters.html')


# Examine Clusters

## Cluster 0

In [81]:
aus_merged.loc[aus_merged['Cluster Labels'] == 0]


,Neighborhood,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
29,"Montopolis, Austin, Texas",0.0,0,30.205513,-97.716575
22,"East Riverside-Oltorf, Austin, Texas",0.0,0,30.216669,-97.685525
23,Easton Park,0.0,0,30.180235,-97.620692
24,"French Place, Austin Texas",0.0,0,30.286838,-97.719248
25,"Galindo, Austin, Texas",0.0,0,30.228714,-97.695736
26,"Granada Hills, Austin, Texas",0.0,0,30.220538,-97.736842
45,"South River City, Austin, Texas",0.0,0,30.254720,-97.744580
28,"Hyde Park (Austin, Texas)",0.0,0,30.179040,-97.799700
47,Swedish Hill Historic District,0.0,0,32.778280,-96.794850
44,"South Lamar, Austin, Texas",0.0,0,30.232261,-97.804078


## Cluster 1

In [82]:
aus_merged.loc[aus_merged['Cluster Labels'] == 1]


,Neighborhood,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
49,"Travis Heights, Austin, Texas",0.02,1,30.243912,-97.742053
59,"Zilker, Austin, Texas",0.04,1,30.251370,-97.766110
14,"Clarksville Historic District (Austin, Texas)",0.03,1,30.281330,-97.762430
36,Old West Austin Historic District,0.02,1,30.274750,-97.761260
58,Zilker Park,0.02,1,30.269140,-97.777270
5,"Bouldin Creek, Austin, Texas",0.02,1,30.252417,-97.757753
18,"Dawson, Austin, Texas",0.02,1,30.260163,-97.755786
40,"Rosedale, Austin, Texas",0.02,1,30.240740,-97.744740


## Cluster 2

In [83]:
aus_merged.loc[aus_merged['Cluster Labels'] == 2]


,Neighborhood,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
2,"Barton Hills, Austin, Texas",0.010000,2,30.255510,-97.783490
56,"Windsor Park, Austin, Texas",0.010000,2,30.286502,-97.756394
55,"Windsor Hills, Austin, Texas",0.010000,2,30.286502,-97.756394
9,"Bryker Woods, Austin, Texas",0.010000,2,30.305032,-97.754192
52,"Westgate, Austin, Texas",0.012048,2,30.187063,-97.836138
38,"Pemberton Heights, Austin, Texas",0.010000,2,30.289817,-97.753361
48,"Tarrytown, Austin, Texas",0.010000,2,30.296150,-97.759690
21,"East Cesar Chavez, Austin, Texas",0.010000,2,30.254640,-97.717565
46,"Spyglass-Barton's Bluff, Austin, Texas",0.010000,2,30.258241,-97.787165
27,"Hancock, Austin, Texas",0.010000,2,30.297198,-97.729140
